# <b>SEC SUSTAINABILITY DISCLOSURE SEARCH<b>

In [1]:
# Imports
# For webscraping, import the following libraries from selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Handle SSL cert

In [2]:
def ssl_cert():
    driver.find_element(By.ID, 'details-button').click()
    driver.find_element(By.ID, 'proceed-link').click()

# Handle Log In

In [3]:
def login():
    # Log in
    # Find the 8th <tr>
    tr = driver.find_elements(By.XPATH, "//tr[@class='odd-row']")
    login = tr[0].find_element(By.XPATH, "//td[8]/a")
    # click the login button
    login.click()
    # wait for the page to load
    time.sleep(3)

    # Find the username and password fields
    username_field = driver.find_element(By.ID, "__ac_name").send_keys("john@smith.com")
    password_field = driver.find_element(By.ID, "__ac_password").send_keys("password")
    # Hit enter to log in
    driver.find_element(By.ID, "__ac_password").send_keys(Keys.ENTER)

# <b>ACCESS THE SITE<b>

In [129]:
# PATHS
CERES_URL = 'https://tools.ceres.org/resources/tools/sec-sustainability-disclosure/'

# import the driver 
driver = webdriver.Chrome()

# access the site 
driver.get(CERES_URL)

# await page load
time.sleep(5)
# handle ssl
ssl_cert()
# await page load
time.sleep(8)
# handle login
login()

In [2]:
# play around
print(driver.title)

Privacy error


# <b>START OF PIPELINE</b>

## Flow of events:
### PREP:
1. Choose Filers
2. Set year to 2009 (default to 2018 at first load)
3. Get number of companies
### COLLECT DATA:
4. Each page shows 100 companies, therefore get total % 100
5. Get all company names per page, add to dictionary, where key = company name, value = count
6. After getting, go to next page (loop starting from 1, and check against each iteration to click data=i+1)
7. Repeat 5-6 till done with last page
### ANNUAL:
8. Repeat 4-7 until no more year, incrementing by 1
9. End

## 0. VARIABLES

In [5]:
ISSUE = 1
YEAR = 2009
TOTAL_YEARS = 15
PAGES = 1
CURRENT_PAGE = 1
NO_OF_COMPANIES = 0

# Create pandas df to store the data, where 1st column is the company name and the rest are the years, from 2009 to 2023
df = pd.DataFrame(columns = ['Company'] + ['Ticker'] + ['Industry Group'] + ['Financial Year End'] + [str(i) for i in range(YEAR, YEAR + TOTAL_YEARS)])

## 1. CHOOSE FILERS

In [5]:
def choose_filers():
    # Find select element where id='searchBoxFilingType'
    searchBoxFilingType = driver.find_element(By.ID, 'searchBoxFilingType')

    # Find the options in the select element
    options = searchBoxFilingType.find_elements(By.TAG_NAME, "option")

    # Select option value='1'
    for option in options:
        if option.get_attribute("value") == '1':
            option.click()
            break

## ISSUE DROPDOWN

In [6]:
# Get the issue dropdown
def choose_issue(issue):
    # Find select element where id='searchBoxIssue'
    searchBoxIssue = driver.find_element(By.ID, 'searchBoxIssue')
    # Find the options in the select element
    options = searchBoxIssue.find_elements(By.TAG_NAME, "option")
    # Select option value = issue
    for option in options:
        if option.get_attribute("value") == str(issue):
            option.click()
            break

In [7]:
# test the function
for i in range(1, 6):
    choose_issue(i)
    time.sleep(3)

## STOCK INDEX DROPDOWN

In [8]:
# Get the stock index dropdown
def choose_stock_index(i):
    # Find select element where id='searchBoxStockIndex'
    searchBoxStockIndex = driver.find_element(By.ID, 'searchBoxStockIndex')
    # Find the options in the select element
    options = searchBoxStockIndex.find_elements(By.TAG_NAME, "option")
    # Click the ith option
    options[i].click()

## 2. SET YEAR

In [10]:
def set_year(year):
    # Set the year
    searchBoxYear = driver.find_element(By.ID, 'searchBoxYear')
    # Find the options in the select element
    options = searchBoxYear.find_elements(By.TAG_NAME, "option")
    # Select the option value = year    
    for option in options:
        if option.get_attribute("value") == str(year):
            option.click()
            break

## 3. GET NUMBER OF COMPANIES

In [11]:
def get_no_of_companies():
    # Find span where id='lblFoundCount'
    NO_OF_COMPANIES = driver.find_element(By.ID, 'lblFoundCount')
    # Split the string to get the number of companies
    NO_OF_COMPANIES = NO_OF_COMPANIES.text.split(' ')[5]
    # Convert the string to integer
    NO_OF_COMPANIES = int(NO_OF_COMPANIES)
    print("Number of companies: ", NO_OF_COMPANIES)
    return NO_OF_COMPANIES

## 4. GET NUMBER OF PAGES

In [12]:
def get_no_of_pages(NO_OF_COMPANIES):
    # Get number of pages: 100 companies per page
    # If number of companies is less than 100, then PAGES = 1
    # If there's remainder, then add 1 to the division
    PAGES = NO_OF_COMPANIES // 100
    if NO_OF_COMPANIES % 100 != 0:
        PAGES += 1
    print("Number of pages: ", PAGES)
    return PAGES

## 5. GET COMPANY NAME PER PAGE

In [13]:
def get_companies(YEAR, df):
    # variables
    added_companies = 0
    
    # EVEN ROWS
    # For each <tr> class with <td>, get the 2nd <td> tag and store it in a list
    company_names_even = driver.find_elements(By.XPATH, "//tr[@class='even-row']")

    # ODD ROWS
    # For each <tr> class with <td>, get the 2nd <td> tag and store it in a list
    company_names_odd = driver.find_elements(By.XPATH, "//tr[@class='odd-row']")
    
    # Combine company names and industry groups
    companies_data = []
    for row in company_names_even + company_names_odd:
        company_name = row.find_element(By.XPATH, ".//td[2]").text  # Extract company name
        ticker = row.find_element(By.XPATH, ".//td[3]").text # Extract ticker
        industry_group = row.find_element(By.XPATH, ".//td[4]").text  # Extract industry group
        issue = row.find_element(By.XPATH, ".//td[6]").text
        stock_index = row.find_element(By.XPATH, ".//td[7]").text
        view_report = row.find_element(By.XPATH, ".//td[8]").text
        companies_data.append([company_name, ticker, industry_group, issue, stock_index, view_report])

    # Sort the companies by name
    companies_data.sort(key=lambda x: x[0])

    # Add the company names to the dataframe, where the first column is the company name and the rest are the years
    for company in companies_data:
        # if the company is not in the dataframe, add it
        if not ((df['Company'] == company[0]) & (df['Issue'] == company[3]) & (df['Stock Index'] == company[4])).any():
            df.loc[len(df)] = [company[0]] + company[1:5] + [0 for i in range(TOTAL_YEARS)]
            # check the view_report value of that company. If it's 'View Report', then add 1 to the year. If it's 'No disclosure', then add 2 to the year
            if company[5] == 'View Report':
                df.loc[df['Company'] == company[0], str(YEAR)] = +2
            else:
                df.loc[df['Company'] == company[0], str(YEAR)] = +1
            added_companies += 1
        # else, update the year
        else:
            if company[5] == 'View Report':
                df.loc[df['Company'] == company[0], str(YEAR)] = +2
            else:   
                df.loc[df['Company'] == company[0], str(YEAR)] = +1

    # count the number of companies
    print(f"{len(companies_data)} found, {added_companies} new companies added!")

# 5. VERSION 2: GET COMPANY NAME PER PAGE (deal with duplicates)

## 6. GO TO NEXT PAGE

In [14]:
# Find the span in the div where id='pagination', and find data=PAGE+1
def get_next_page(CURRENT_PAGE):
    next_page_str = str(CURRENT_PAGE)
    next_page = driver.find_element(By.XPATH, f".//span[@class='page ' and @data={next_page_str}]")
    next_page.click()

# CURRENT_PAGE += 1
# get_next_page(CURRENT_PAGE)

# <b>RUN EVERYTHING</b>

In [15]:
# RESET EVERYTHING
ISSUE = 1
YEAR = 2009
TOTAL_YEARS = 15
PAGES = 1
CURRENT_PAGE = 1
NO_OF_COMPANIES = 0

# Create pandas df to store the data, where 1st column is the company name and the rest are the years, from 2009 to 2023
df = pd.DataFrame(columns = ['Company'] + ['Ticker'] + ['Industry Group'] + ['Issue'] + ['Stock Index'] + [str(i) for i in range(YEAR, YEAR + TOTAL_YEARS)])

# VERSION 1: Each Index

In [70]:
# 1) Choose filers
choose_filers()

YEAR = 2009

# # set the first page
# first_page = driver.find_element(By.XPATH, ".//span[@class='page ' and @data='1']")
# first_page.click()

# Loop through the issue
for i in range(1, 6):
    print("=====================================")
    # lazy stuff
    issue_text = ""
    if i == 1:
        issue_text = "Climate Change"
    elif i == 2:
        issue_text = "Hydraulic Fracturing"
    elif i == 3:
        issue_text = "Water Risk"
    elif i == 4:
        issue_text = "Carobon Asset Risk"
    elif i == 5:
        issue_text = "Human & Workers' Rights"
    print("CURRENT ISSUE: ", issue_text)
    # Reset the year
    YEAR = 2009
    
    choose_issue(i)
    time.sleep(1)
    
    # Loop through the stock index
    for j in range(1, 4):
        # lazy stuff
        stock_index_text = ""
        
        if j == 1:
            stock_index_text = "S&P 500"
        elif j == 2:
            stock_index_text = "Russell 3000"
        elif j == 3:
            stock_index_text = "FT Global 500"
        print("CURRENT STOCK INDEX: ", stock_index_text)
        choose_stock_index(j)
        time.sleep(1)
        
        # Reset the year
        YEAR = 2009
        
        # Loop through the years
        # for i in range(2):
        for i in range(YEAR, (YEAR + TOTAL_YEARS)):
            print("CURRENT YEAR: ", YEAR)
            # 2) Set Year
            set_year(YEAR)
            # WAIT 5 SECONDS
            time.sleep(15)

            CURRENT_PAGE = 1

            # 3) Get number of companies
            NO_OF_COMPANIES = get_no_of_companies()
            # 4) Get number of pages
            PAGES = get_no_of_pages(NO_OF_COMPANIES)


            # LOOP THROUGH
            for i in range(PAGES):
                print("Current page: ", CURRENT_PAGE)
                # 5) Get company name per page ***
                get_companies(YEAR)
                # 6) Go to next page
                CURRENT_PAGE += 1
                
                # if current page is greater than the number of pages, then break
                if CURRENT_PAGE > PAGES:
                    print("No more pages!")
                    print()
                    break
                else:
                    get_next_page(CURRENT_PAGE)
                print()
                
            # Increment YEAR
            YEAR += 1
            
        print("=====================================")
        print()


CURRENT ISSUE:  Climate Change
CURRENT STOCK INDEX:  S&P 500
CURRENT YEAR:  2009
Number of companies:  494
Number of pages:  5
Current page:  1
100 found, 0 new companies added!

Current page:  2
100 found, 0 new companies added!

Current page:  3
100 found, 0 new companies added!

Current page:  4
100 found, 0 new companies added!

Current page:  5
94 found, 0 new companies added!
No more pages!

CURRENT YEAR:  2010
Number of companies:  498
Number of pages:  5
Current page:  1
100 found, 0 new companies added!

Current page:  2
100 found, 0 new companies added!

Current page:  3
100 found, 0 new companies added!

Current page:  4
100 found, 0 new companies added!

Current page:  5
98 found, 0 new companies added!
No more pages!

CURRENT YEAR:  2011
Number of companies:  506
Number of pages:  6
Current page:  1
100 found, 0 new companies added!

Current page:  2
100 found, 0 new companies added!

Current page:  3
100 found, 0 new companies added!

Current page:  4
100 found, 0 new com

WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: chrome=125.0.6422.114)
Stacktrace:
0   chromedriver                        0x0000000100bf24c8 chromedriver + 4302024
1   chromedriver                        0x0000000100beae10 chromedriver + 4271632
2   chromedriver                        0x000000010081c000 chromedriver + 278528
3   chromedriver                        0x0000000100806a28 chromedriver + 191016
4   chromedriver                        0x000000010080485c chromedriver + 182364
5   chromedriver                        0x0000000100805310 chromedriver + 185104
6   chromedriver                        0x0000000100813620 chromedriver + 243232
7   chromedriver                        0x0000000100896ad8 chromedriver + 781016
8   chromedriver                        0x0000000100853004 chromedriver + 503812
9   chromedriver                        0x00000001008539ec chromed

#### Continue where you left off

In [76]:
# 1) Choose filers
choose_filers()

YEAR = 2022

# # set the first page
# first_page = driver.find_element(By.XPATH, ".//span[@class='page ' and @data='1']")
# first_page.click()

# Loop through the issue
# for i in range(2, 6):
#     print("=====================================")
#     # lazy stuff
#     issue_text = ""
#     if i == 1:
#         issue_text = "Climate Change"
#     elif i == 2:
#         issue_text = "Hydraulic Fracturing"
#     elif i == 3:
#         issue_text = "Water Risk"
#     elif i == 4:
#         issue_text = "Carobon Asset Risk"
#     elif i == 5:
#         issue_text = "Human & Workers' Rights"
#     print("CURRENT ISSUE: ", issue_text)
#     # Reset the year
#     YEAR = 2009
    
choose_issue(5)
time.sleep(1)

# Loop through the stock index
for j in range(3, 4):
    # lazy stuff
    stock_index_text = ""
    
    if j == 1:
        stock_index_text = "S&P 500"
    elif j == 2:
        stock_index_text = "Russell 3000"
    elif j == 3:
        stock_index_text = "FT Global 500"
    print("CURRENT STOCK INDEX: ", stock_index_text)
    choose_stock_index(j)
    time.sleep(1)
    
    # Reset the year
    YEAR = 2022
    
    # Loop through the years
    for i in range(2):
    # for i in range(YEAR, (YEAR + TOTAL_YEARS)):
        print("CURRENT YEAR: ", YEAR)
        # 2) Set Year
        set_year(YEAR)
        # WAIT 5 SECONDS
        time.sleep(10)

        CURRENT_PAGE = 1

        # 3) Get number of companies
        NO_OF_COMPANIES = get_no_of_companies()
        # 4) Get number of pages
        PAGES = get_no_of_pages(NO_OF_COMPANIES)


        # LOOP THROUGH
        for i in range(PAGES):
            print("Current page: ", CURRENT_PAGE)
            # 5) Get company name per page ***
            get_companies(YEAR)
            # 6) Go to next page
            CURRENT_PAGE += 1
            
            # if current page is greater than the number of pages, then break
            if CURRENT_PAGE > PAGES:
                print("No more pages!")
                print()
                break
            else:
                get_next_page(CURRENT_PAGE)
            time.sleep(1)
            print()
            
        # Increment YEAR
        YEAR += 1
        
    print("=====================================")
    print()


CURRENT STOCK INDEX:  FT Global 500
CURRENT YEAR:  2022
Number of companies:  178
Number of pages:  2
Current page:  1
100 found, 3 new companies added!

Current page:  2
78 found, 1 new companies added!
No more pages!

CURRENT YEAR:  2023
Number of companies:  140
Number of pages:  2
Current page:  1
100 found, 4 new companies added!

Current page:  2
40 found, 0 new companies added!
No more pages!




## TEST

In [104]:
# RESET EVERYTHING
ISSUE = 1
YEAR = 2009
TOTAL_YEARS = 15
PAGES = 1
CURRENT_PAGE = 1
NO_OF_COMPANIES = 0

# Create pandas df to store the data, where 1st column is the company name and the rest are the years, from 2009 to 2023
test_df = pd.DataFrame(columns = ['Company'] + ['Ticker'] + ['Industry Group'] + ['Issue'] + ['Stock Index'] + [str(i) for i in range(YEAR, YEAR + TOTAL_YEARS)])

# 1) Choose filers
choose_filers()

YEAR = 2013

# # set the first page
# first_page = driver.find_element(By.XPATH, ".//span[@class='page ' and @data='1']")
# first_page.click()

# Loop through the issue
# for i in range(2, 6):
print("=====================================")
#     # lazy stuff
#     issue_text = ""
#     if i == 1:
#         issue_text = "Climate Change"
#     elif i == 2:
#         issue_text = "Hydraulic Fracturing"
#     elif i == 3:
#         issue_text = "Water Risk"
#     elif i == 4:
#         issue_text = "Carobon Asset Risk"
#     elif i == 5:
#         issue_text = "Human & Workers' Rights"
#     print("CURRENT ISSUE: ", issue_text)
#     # Reset the year
#     YEAR = 2009
    
choose_issue(1)
time.sleep(1)

# # Loop through the stock index
# for j in range(3, 4):
#     # lazy stuff
#     stock_index_text = ""
    
#     if j == 1:
#         stock_index_text = "S&P 500"
#     elif j == 2:
#         stock_index_text = "Russell 3000"
#     elif j == 3:
#         stock_index_text = "FT Global 500"
# print("CURRENT STOCK INDEX: ", stock_index_text)
choose_stock_index(0)
time.sleep(1)

# Reset the year
YEAR = 2013

# Loop through the years
for i in range(1):
# for i in range(YEAR, (YEAR + TOTAL_YEARS)):
    print("CURRENT YEAR: ", YEAR)
    # 2) Set Year
    set_year(YEAR)
    # WAIT 5 SECONDS
    time.sleep(10)

    CURRENT_PAGE = 1

    # 3) Get number of companies
    NO_OF_COMPANIES = get_no_of_companies()
    # 4) Get number of pages
    PAGES = get_no_of_pages(NO_OF_COMPANIES)


    # LOOP THROUGH
    for i in range(PAGES):
        print("Current page: ", CURRENT_PAGE)
        # 5) Get company name per page ***
        get_companies(YEAR)
        # 6) Go to next page
        CURRENT_PAGE += 1
        
        # if current page is greater than the number of pages, then break
        if CURRENT_PAGE > PAGES:
            print("No more pages!")
            print()
            break
        else:
            get_next_page(CURRENT_PAGE)
        time.sleep(1)
        print()
        
    # Increment YEAR
    YEAR += 1
    
print("=====================================")
print()


CURRENT YEAR:  2013
Number of companies:  4755
Number of pages:  48
Current page:  1
100 found, 99 new companies added!

Current page:  2
100 found, 99 new companies added!

Current page:  3
100 found, 99 new companies added!

Current page:  4
100 found, 99 new companies added!

Current page:  5
100 found, 100 new companies added!

Current page:  6
100 found, 100 new companies added!

Current page:  7
100 found, 100 new companies added!

Current page:  8
100 found, 99 new companies added!

Current page:  9
100 found, 99 new companies added!

Current page:  10
100 found, 99 new companies added!

Current page:  11
100 found, 100 new companies added!

Current page:  12
100 found, 99 new companies added!

Current page:  13
100 found, 100 new companies added!

Current page:  14
100 found, 99 new companies added!

Current page:  15
100 found, 100 new companies added!

Current page:  16
100 found, 100 new companies added!

Current page:  17
100 found, 99 new companies added!

Current page:  1

# VERSION 2: Index = All companies

In [ ]:
# RESET EVERYTHING
ISSUE = 1
YEAR = 2009
TOTAL_YEARS = 15
PAGES = 1
CURRENT_PAGE = 1
NO_OF_COMPANIES = 0

# Create pandas df to store the data, where 1st column is the company name and the rest are the years, from 2009 to 2023
all_index_df = pd.DataFrame(columns = ['Company'] + ['Ticker'] + ['Industry Group'] + ['Issue'] + ['Stock Index'] + [str(i) for i in range(YEAR, YEAR + TOTAL_YEARS)])

# 1) Choose filers
choose_filers()
# 2) Choose stock index
choose_stock_index(0)
time.sleep(1)

# # set the first page
# first_page = driver.find_element(By.XPATH, ".//span[@class='page ' and @data='1']")
# first_page.click()

# Loop through the issue
for i in range(1, 6):
    print("=====================================")
    # lazy stuff
    issue_text = ""
    if i == 1:
        issue_text = "Climate Change"
    elif i == 2:
        issue_text = "Hydraulic Fracturing"
    elif i == 3:
        issue_text = "Water Risk"
    elif i == 4:
        issue_text = "Carobon Asset Risk"
    elif i == 5:
        issue_text = "Human & Workers' Rights"
    print("CURRENT ISSUE: ", issue_text)
    print("CURRENT STOCK INDEX: ", "All Companies")
    # Reset the year
    YEAR = 2009
    
    # Choose issue
    choose_issue(i)
    time.sleep(1)

    # Reset the year
    YEAR = 2009

    # Loop through the years
    # for j in range(1):
    for j in range(YEAR, (YEAR + TOTAL_YEARS)):
        print("CURRENT YEAR: ", YEAR)
        # 2) Set Year
        set_year(YEAR)
        # WAIT 5 SECONDS
        time.sleep(10)

        CURRENT_PAGE = 1

        # 3) Get number of companies
        NO_OF_COMPANIES = get_no_of_companies()
        # 4) Get number of pages
        PAGES = get_no_of_pages(NO_OF_COMPANIES)

        # Loop through the pages
        for k in range(PAGES):
            print("Current page: ", CURRENT_PAGE)
            # 5) Get company name per page ***
            get_companies(YEAR, all_index_df)
            # 6) Go to next page
            CURRENT_PAGE += 1
            
            # if current page is greater than the number of pages, then break
            if CURRENT_PAGE > PAGES:
                print("No more pages!")
                print()
                break
            else:
                get_next_page(CURRENT_PAGE)
            time.sleep(1)
            print()
            
        # Increment YEAR
        YEAR += 1
        
    print("=====================================")
    print()


## PROCESS DATA

#### Count each Year rows

In [51]:
# in df, count columns '2009' and '2010' values that != 0
count_2009 = df['2009'].value_counts()
count_2010 = df['2010'].value_counts()
count_2011 = df['2011'].value_counts()
count_2012 = df['2012'].value_counts()
count_2013 = df['2013'].value_counts()
count_2014 = df['2014'].value_counts()
count_2015 = df['2015'].value_counts()
count_2016 = df['2016'].value_counts()
count_2017 = df['2017'].value_counts()
count_2018 = df['2018'].value_counts()
count_2019 = df['2019'].value_counts()
count_2020 = df['2020'].value_counts()
count_2021 = df['2021'].value_counts()
count_2022 = df['2022'].value_counts()
count_2023 = df['2023'].value_counts()

print("2009: ", count_2009[1] + count_2009[2])
print("2010: ", count_2010[1] + count_2010[2])
print("2011: ", count_2011[1] + count_2011[2])
print("2012: ", count_2012[1] + count_2012[2])
print("2013: ", count_2013[1] + count_2013[2])
print("2014: ", count_2014[1] + count_2014[2])
print("2015: ", count_2015[1] + count_2015[2])
print("2016: ", count_2016[1] + count_2016[2])
print("2017: ", count_2017[1] + count_2017[2])
print("2018: ", count_2018[1] + count_2018[2])
print("2019: ", count_2019[1] + count_2019[2])
print("2020: ", count_2020[1] + count_2020[2])
print("2021: ", count_2021[1] + count_2021[2])
print("2022: ", count_2022[1] + count_2022[2])
print("2023: ", count_2023[1] + count_2023[2])


2009:  494
2010:  498
2011:  506
2012:  512
2013:  523
2014:  533
2015:  543
2016:  547
2017:  550
2018:  547
2019:  509
2020:  516
2021:  498
2022:  514
2023:  391


# Save to CSV

In [ ]:
# Save the dataframe to a csv file
df.to_csv('ceres_years_data.csv', index=False)

In [77]:
# Save the full version to csv
df.to_csv('ceres_full_data.csv', index=False)

# ====================================================================

# <B>PART 2: EXTRACT FROM REPORT<B>
# ====================================================================

# <b>START OF PIPELINE</b>

## Flow of events:
### PREP:
1. Choose Filers
2. Set year to 2009 (default to 2018 at first load)
3. Get number of companies
### COLLECT DATA:
4. For each Issue, do the following:\
    5. For each Year, do the following:\
        6. Each page shows 100 companies, therefore get total number of pages from that\
        7. Get all company names per page, then access report\
            &nbsp;&nbsp;&nbsp;&nbsp;a. Get each row from page\
            &nbsp;&nbsp;&nbsp;&nbsp;b. Add to dataframe\
            &nbsp;&nbsp;&nbsp;&nbsp;c. Check if Report column is 'Non Disclosure' or 'View Report'\
            &nbsp;&nbsp;&nbsp;&nbsp;d. If 'View Report', do the following:\
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;i. Click the link\
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ii. Click 'View Extended Disclosures' (if not clicked already)\
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;iii. Get all excerpts\
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;iv. Count the total words, and the keywords (Green, Climate, Sustainability)\
            &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;v. Add to dataframe\
            &nbsp;&nbsp;&nbsp;&nbsp;e. Else, put '-' at in the columns [Green, Climate, Sustainability], and put 'No Disclosure'\
        8. After getting, go to next page (loop starting from 1, and check against each iteration to click data=i+1)\
        9. Repeat 5-6 till done with last page\
10. End 

# 1. ACCESS REPORT

In [136]:
def get_report(YEAR, df):    
    # EVEN ROWS
    # For each <tr> class with <td>, get the 2nd <td> tag and store it in a list
    company_names_even = driver.find_elements(By.XPATH, "//tr[@class='even-row']")

    # ODD ROWS
    # For each <tr> class with <td>, get the 2nd <td> tag and store it in a list
    company_names_odd = driver.find_elements(By.XPATH, "//tr[@class='odd-row']")
    
    # Combine even and odd rows
    company_names = company_names_even + company_names_odd
    # Sort the companies by name
    company_names.sort(key=lambda x: x.find_element(By.XPATH, ".//td[2]").text)
    
    # Combine company names and industry groups
    companies_data = []
    for row in company_names:
        company_name = row.find_element(By.XPATH, ".//td[2]").text  # Extract company name
        ticker = row.find_element(By.XPATH, ".//td[3]").text # Extract ticker
        industry_group = row.find_element(By.XPATH, ".//td[4]").text  # Extract industry group
        issue = row.find_element(By.XPATH, ".//td[6]").text
        stock_index = row.find_element(By.XPATH, ".//td[7]").text
        # if stock_index is empty, then put '-'
        if stock_index == '':
            stock_index = '-'
        view_report = row.find_element(By.XPATH, ".//td[8]").text
        year = YEAR
        
        # check the view_report value of that company. If it's 'View Report', then click the link and extract the data
        # Click the 'View Report' link
        if view_report == 'View Report':
            # click the link
            row.find_element(By.XPATH, ".//td[8]/a").click()
            # wait for the page to load
            time.sleep(4)
            
            # see if it successfully loaded by checking a div's style, namely div with id='secAbstractDetailsOverlay' with style='display: block;'
            if driver.find_element(By.ID, 'secAbstractDetailsOverlay').get_attribute('style') == 'display: block;':
                # if text of id='toggleExtended' is 'View Extended Disclosures', then click it
                if driver.find_element(By.ID, 'toggleExtended').text == 'View Extended Disclosures':
                    driver.find_element(By.ID, 'toggleExtended').click()
                    # wait for the page to load
                    time.sleep(2)
                
                # data is in even/odd rows, but the class of each text is still 'excerpt'. So, we can use that to extract the data
                # find all the excerpts
                excerpts = driver.find_elements(By.CLASS_NAME, 'excerpt')
                # extract the data
                data = [excerpt.text for excerpt in excerpts]
                # count the words in the data
                word_count = len(' '.join(data).split())
                # get the count of the following words: 'green', 'climate', 'sustain' and lower the case
                green_count = ' '.join(data).lower().count('green')
                climate_count = ' '.join(data).lower().count('climate')
                sustain_count = ' '.join(data).lower().count('sustain')
                
                # print the data
                print('-----------------------------------')
                print('Company: ', company_name)
                print('Year: ', year)
                print('Green: ', green_count)
                print('Climate: ', climate_count)
                print('Sustain: ', sustain_count)
                print('Total Words: ', word_count)
                print()
                
                # click the 'Close' button
                driver.find_element(By.ID, 'boxclose').click()
                # wait for the page to load
                time.sleep(1)
            
            # else, say that the page did not load and add 'Error Loading Page' to the 'Total Words' column, and put '-' in the 'Green', 'Climate' and 'Sustain' columns
            else:
                word_count = 'Error Loading Page'
                green_count = '-'
                climate_count = '-'
                sustain_count = '-'
                
                # print 'Error Loading Page'
                print('-----------------------------------')
                print('Company: ', company_name)
                print('ERROR LOADING PAGE')
                print()
        
        # else, put 'No Disclosure' in the 'Total Words' column, and put '-' in the 'Green', 'Climate' and 'Sustain' columns
        else:
            word_count = 'No Disclosure'
            green_count = '-'
            climate_count = '-'
            sustain_count = '-'
            
            # print 'No Disclosure'
            print('-----------------------------------')
            print('Company: ', company_name)
            print('NO DISCLOSURE')
            print()
        
        companies_data.append([company_name, ticker, industry_group, issue, stock_index, year, green_count, climate_count, sustain_count,  word_count, view_report])

    # # Sort the companies by name
    # companies_data.sort(key=lambda x: x[0])

    # Add the company names to the dataframe, where the first column is the company name and the rest are the years
    for company in companies_data:
        # # debug: print the company
        # print("DEBUG: Company: ", company[0])
        # for i in range(len(company)):
        #     print(company[i])
        
        # if the company is not in the dataframe, add it
        if not ((df['Company'] == company[0]) & 
                (df['Issue'] == company[3]) & 
                (df['Stock Index'] == company[4]) & 
                (df['Year'] == company[5])).any():
            
            # Create a new row with the necessary information and initialize Green, Climate, Sustain to 0
            new_row = pd.DataFrame([{
                'Company': company[0],
                'Ticker': company[1],
                'Industry Group': company[2],
                'Issue': company[3],
                'Stock Index': company[4],
                'Year': company[5],
                'Green': company[6],
                'Climate': company[7],
                'Sustain': company[8],
                'Total Words': company[9]
            }])
            
            # Append the new row to the DataFrame using pd.concat
            df = pd.concat([df, new_row], ignore_index=True)
            
            # Print that the company has been added
            print(f"{company[0]} added to the DataFrame!")
            
    # Return the DataFrame
    return df

# <b>RUN THE PIPELINE<b>

In [131]:
# RESET EVERYTHING
ISSUE = 1
YEAR = 2009
TOTAL_YEARS = 15
PAGES = 1
CURRENT_PAGE = 1
NO_OF_COMPANIES = 0

# Create pandas df to store the data, where 1st column is the company name and the rest are the years, from 2009 to 2023
report_df = pd.DataFrame(columns = ['Company'] + ['Ticker'] + ['Industry Group'] + ['Issue'] + ['Stock Index'] + ['Year'] + ['Green'] + ['Climate'] + ['Sustain'] + ['Total Words'])

In [135]:
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException

# START THE PROCESS
print('===============================================')

# 1) Choose filers
choose_filers()
# 2) Choose stock index
choose_stock_index(0)

# TEST: do for 1 year, 2009
YEAR = 2022
# 2) Set Year
set_year(YEAR)
# WAIT 5 SECONDS
time.sleep(5)

CURRENT_PAGE = 1

# 3) Get number of companies
NO_OF_COMPANIES = get_no_of_companies()
# 4) Get number of pages
PAGES = get_no_of_pages(NO_OF_COMPANIES)

# Loop through the issues
for i in range(1, 6):
    # lazy stuff
    issue_text = ""
    if i == 1:
        issue_text = "Climate Change"
    elif i == 2:
        issue_text = "Hydraulic Fracturing"
    elif i == 3:
        issue_text = "Water Risk"
    elif i == 4:
        issue_text = "Carobon Asset Risk"
    elif i == 5:
        issue_text = "Human & Workers' Rights"
    print("CURRENT ISSUE: ", issue_text)
    # Reset the year
    YEAR = 2022
    
    choose_issue(i)
    time.sleep(2)
    
    # Loop through the pages
    for i in range(PAGES):
        print("Current page: ", CURRENT_PAGE)
        # 5) Get report per page ***
        # handle StaleElementReferenceException
        while True:
            try:
                # 5) Get company name per page ***
                report_df = get_report(YEAR, report_df)
                break
            except StaleElementReferenceException:
                print("StaleElementReferenceException, retrying...")
                time.sleep(1)
                
                # check if the close button is present
                try:
                    driver.find_element(By.ID, 'boxclose').click()
                    time.sleep(1)
                except NoSuchElementException:
                    print("No close button found! Continuing...")
                except ElementNotInteractableException:
                    print("No close button found! Continuing...")
                except StaleElementReferenceException:
                    print("No close button found! Continuing...")
                    
                continue
            except NoSuchElementException:
                print("NoSuchElementException, retrying...")
                # refresh the page
                driver.refresh()
                # wait for the page to load
                time.sleep(8)
                # set everything again
                choose_filers()
                choose_stock_index(0)
                set_year(YEAR)
                time.sleep(6)
                # go to the page again
                # handle NoSuchElementException
                try:
                    get_next_page(CURRENT_PAGE)
                    continue
                except NoSuchElementException:
                    # if the page is not found, that means we are already on the CURRENT_PAGE
                    print("Page exception: Already on the page! Continuing...")
                except ElementClickInterceptedException:
                    print("ElementClickInterceptedException, retrying...")
                    time.sleep(1)
                    continue
                except StaleElementReferenceException:
                    print("StaleElementReferenceException, retrying...")
                    time.sleep(1)
                    continue
                continue
            # handle except ElementNotInteractableException or ElementClickInterceptedException
            except ElementNotInteractableException:
                print("ElementNotInteractableException, retrying...")
                # check if the close button is present
                try:
                    driver.find_element(By.ID, 'boxclose').click()
                    time.sleep(1)
                except NoSuchElementException:
                    print("No close button found! Continuing...")
                except ElementNotInteractableException:
                    print("No close button found! Continuing...")
                continue
            except ElementClickInterceptedException:
                print("ElementClickInterceptedException, retrying...")
                # check if the close button is present
                try:
                    driver.find_element(By.ID, 'boxclose').click()
                    time.sleep(1)
                except NoSuchElementException:
                    print("No close button found! Continuing...")
                except ElementNotInteractableException:
                    print("No close button found! Continuing...")
                except StaleElementReferenceException:
                    print("No close button found! Continuing...")
                continue
        
            
        # 6) Go to next page
        CURRENT_PAGE += 1

        # if current page is greater than the number of pages, then break
        if CURRENT_PAGE > PAGES:
            print("No more pages!")
            print()
            break
        else:
            try:
                get_next_page(CURRENT_PAGE)
                time.sleep(1)
            except ElementClickInterceptedException:
                print("ElementClickInterceptedException, retrying...")
                time.sleep(1)
                continue
        print()


Number of companies:  4666
Number of pages:  47
CURRENT ISSUE:  Climate Change
Current page:  1
-----------------------------------
Company:  1 800 FLOWERS COM INC
Year:  2022
Green:  0
Climate:  0
Sustain:  0
Total Words:  335

-----------------------------------
Company:  10X GENOMICS, INC.
NO DISCLOSURE

-----------------------------------
Company:  180 LIFE SCIENCES CORP.
NO DISCLOSURE

-----------------------------------
Company:  1LIFE HEALTHCARE INC
NO DISCLOSURE

-----------------------------------
Company:  1ST SOURCE CORP
Year:  2022
Green:  0
Climate:  5
Sustain:  0
Total Words:  496

-----------------------------------
Company:  1STDIBS.COM, INC.
NO DISCLOSURE

-----------------------------------
Company:  22ND CENTURY GROUP, INC.
Year:  2022
Green:  0
Climate:  0
Sustain:  0
Total Words:  98

-----------------------------------
Company:  2U, INC.
NO DISCLOSURE

-----------------------------------
Company:  374WATER INC.
Year:  2022
Green:  1
Climate:  1
Sustain:  2
Total W

ElementClickInterceptedException: Message: element click intercepted: Element <span class="page " data="...">9</span> is not clickable at point (174, 7). Other element would receive the click: <div class="resolution-filters" id="searchFilters">...</div>
  (Session info: chrome=125.0.6422.142)
Stacktrace:
0   chromedriver                        0x00000001007ba4c8 chromedriver + 4302024
1   chromedriver                        0x00000001007b2e10 chromedriver + 4271632
2   chromedriver                        0x00000001003e419c chromedriver + 278940
3   chromedriver                        0x000000010042bd48 chromedriver + 572744
4   chromedriver                        0x000000010042a2ec chromedriver + 565996
5   chromedriver                        0x000000010042847c chromedriver + 558204
6   chromedriver                        0x00000001004279cc chromedriver + 555468
7   chromedriver                        0x000000010041ce24 chromedriver + 511524
8   chromedriver                        0x00

#### Continue where you left off

In [140]:
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException

# START THE PROCESS
print('===============================================')

# 1) Choose filers
choose_filers()
# 2) Choose stock index
choose_stock_index(0)

# TEST: do for 1 year, 2009
YEAR = 2022
# 2) Set Year
set_year(YEAR)
# WAIT 5 SECONDS
time.sleep(5)

CURRENT_PAGE = 32

# 3) Get number of companies
NO_OF_COMPANIES = get_no_of_companies()
# 4) Get number of pages
PAGES = get_no_of_pages(NO_OF_COMPANIES)

# Loop through the issues
for i in range(1, 6):
    # lazy stuff
    issue_text = ""
    if i == 1:
        issue_text = "Climate Change"
    elif i == 2:
        issue_text = "Hydraulic Fracturing"
    elif i == 3:
        issue_text = "Water Risk"
    elif i == 4:
        issue_text = "Carobon Asset Risk"
    elif i == 5:
        issue_text = "Human & Workers' Rights"
    print("CURRENT ISSUE: ", issue_text)
    # Reset the year
    YEAR = 2022
    
    choose_issue(i)
    time.sleep(2)
    
    # Loop through the pages
    for i in range(PAGES - 8):
        print("Current page: ", CURRENT_PAGE)
        # 5) Get report per page ***
        # handle StaleElementReferenceException
        while True:
            try:
                # 5) Get company name per page ***
                report_df = get_report(YEAR, report_df)
                break
            except StaleElementReferenceException:
                print("StaleElementReferenceException, retrying...")
                time.sleep(1)
                
                # check if the close button is present
                try:
                    driver.find_element(By.ID, 'boxclose').click()
                    time.sleep(1)
                except NoSuchElementException:
                    print("No close button found! Continuing...")
                except ElementNotInteractableException:
                    print("No close button found! Continuing...")
                except StaleElementReferenceException:
                    print("No close button found! Continuing...")
                    
                continue
            except NoSuchElementException:
                print("NoSuchElementException, retrying...")
                # refresh the page
                driver.refresh()
                # wait for the page to load
                time.sleep(8)
                # set everything again
                choose_filers()
                choose_stock_index(0)
                set_year(YEAR)
                time.sleep(6)
                # go to the page again
                # handle NoSuchElementException
                try:
                    get_next_page(CURRENT_PAGE)
                    continue
                except NoSuchElementException:
                    # if the page is not found, that means we are already on the CURRENT_PAGE
                    print("Page exception: Already on the page! Continuing...")
                except ElementClickInterceptedException:
                    print("ElementClickInterceptedException, retrying...")
                    time.sleep(1)
                    continue
                except StaleElementReferenceException:
                    print("StaleElementReferenceException, retrying...")
                    time.sleep(1)
                    continue
                continue
            # handle except ElementNotInteractableException or ElementClickInterceptedException
            except ElementNotInteractableException:
                print("ElementNotInteractableException, retrying...")
                # check if the close button is present
                try:
                    driver.find_element(By.ID, 'boxclose').click()
                    time.sleep(1)
                except NoSuchElementException:
                    print("No close button found! Continuing...")
                except ElementNotInteractableException:
                    print("No close button found! Continuing...")
                continue
            except ElementClickInterceptedException:
                print("ElementClickInterceptedException, retrying...")
                # check if the close button is present
                try:
                    driver.find_element(By.ID, 'boxclose').click()
                    time.sleep(1)
                except NoSuchElementException:
                    print("No close button found! Continuing...")
                except ElementNotInteractableException:
                    print("No close button found! Continuing...")
                except StaleElementReferenceException:
                    print("No close button found! Continuing...")
                continue
        
            
        # 6) Go to next page
        CURRENT_PAGE += 1

        # if current page is greater than the number of pages, then break
        if CURRENT_PAGE > PAGES:
            print("No more pages!")
            print()
            break
        else:
            try:
                get_next_page(CURRENT_PAGE)
                time.sleep(1)
            except ElementClickInterceptedException:
                print("ElementClickInterceptedException, retrying...")
                time.sleep(1)
                continue
        print()


Number of companies:  4666
Number of pages:  47
CURRENT ISSUE:  Climate Change
Current page:  32
NoSuchElementException, retrying...
-----------------------------------
Company:  ONEOK INC /NEW/
Year:  2022
Green:  2
Climate:  22
Sustain:  1
Total Words:  5765

-----------------------------------
Company:  ONESPAN INC.
Year:  2022
Green:  0
Climate:  3
Sustain:  0
Total Words:  122

-----------------------------------
Company:  ONESPAWORLD HOLDINGS LTD
Year:  2022
Green:  2
Climate:  5
Sustain:  0
Total Words:  903

-----------------------------------
Company:  ONTO INNOVATION INC.
Year:  2022
Green:  0
Climate:  2
Sustain:  0
Total Words:  182

-----------------------------------
Company:  ONTRAK, INC.
NO DISCLOSURE

-----------------------------------
Company:  OOMA INC
Year:  2022
Green:  0
Climate:  4
Sustain:  0
Total Words:  97

-----------------------------------
Company:  OP BANCORP
Year:  2022
Green:  0
Climate:  1
Sustain:  0
Total Words:  34

--------------------------------

KeyboardInterrupt: 

In [ ]:
import pandas as pd

# import csv as df
data = pd.read_csv('ceres_years_data.csv')

In [141]:
# Save the full version to csv
report_df.to_csv('ceres_words_partial.csv', index=False)